In [1]:
import numpy as np
import csv
import re
from sympy import Matrix
from sympy import init_printing

# Initialiser l'affichage de sympy pour une meilleure présentation
init_printing()

# Lire le fichier CSV et convertir les données en matrice
with open('Matrices/K.csv', encoding='utf-8-sig') as fichier_csv:
    lecteur_csv = csv.reader(fichier_csv)
    
    # Convertir en matrice (liste de listes)
    K = [ligne[:18] for ligne in list(lecteur_csv)[:18]]
    K = [[float(cellule) for cellule in ligne] for ligne in K]

# Convertir la liste de listes en une matrice sympy
K_sympy = Matrix(K)

# Afficher la matrice sympy
K_sympy


⎡89380.0   -89380.0       0.0             0.0           0.0         0.0        ↪
⎢                                                                              ↪
⎢-89380.0  96880.0      -7500.0           0.0           0.0         0.0        ↪
⎢                                                                              ↪
⎢  0.0     -7500.0   10000000000.0   -10000000000.0     0.0         0.0        ↪
⎢                                                                              ↪
⎢  0.0       0.0     -10000000000.0  10000000000.0   -3872000.0     0.0        ↪
⎢                                                                              ↪
⎢  0.0       0.0          0.0          -3872000.0    6343000.0   -2471000.0    ↪
⎢                                                                              ↪
⎢  0.0       0.0          0.0             0.0        -2471000.0  4942000.0   - ↪
⎢                                                                              ↪
⎢  0.0       0.0          0.